# **Bitcoin price prediction - Linear Regression**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it


---


Description: executing the chosen model, first with default values, then by choosing the best parameters by performing hyperparameter tuning with cross validation and performance evaluation.

# Global constants, dependencies, libraries and tools

In [1]:
# Main constants
LOCAL_RUNNING = True
SLOW_OPERATIONS = True # Decide whether or not to use operations that might slow down notebook execution
SPLITTING_METHOD = "walk_forward_splits"
MODEL_NAME = "LinearRegression"
ROOT_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [2]:
if not LOCAL_RUNNING: 
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(ROOT_DIR, force_remount=True)

In [3]:
# Set main dir
MAIN_DIR = ROOT_DIR + "" if LOCAL_RUNNING else ROOT_DIR + "/MyDrive/BDC/project"

###################
# --- DATASET --- #
###################

# Datasets dirs
DATASET_OUTPUT_DIR = MAIN_DIR + "/datasets/output"

# Datasets names
DATASET_TRAIN_VALID_NAME = "bitcoin_blockchain_data_15min_train_valid"

# Datasets paths
DATASET_TRAIN_VALID  = DATASET_OUTPUT_DIR + "/" + DATASET_TRAIN_VALID_NAME + ".parquet"

####################
# --- FEATURES --- #
####################

# Features dir
FEATURES_DIR = MAIN_DIR + "/features"

# Features labels
FEATURES_LABEL = "features"
TARGET_LABEL = "next-market-price"

# Features names
ALL_FEATURES_NAME = "all_features"
MOST_REL_FEATURES_NAME = "most_rel_features"
LEAST_REL_FEATURES_NAME = "least_rel_features"

# Features paths
ALL_FEATURES = FEATURES_DIR + "/" + ALL_FEATURES_NAME + ".json"
MOST_REL_FEATURES = FEATURES_DIR + "/" + MOST_REL_FEATURES_NAME + ".json"
LEAST_REL_FEATURES = FEATURES_DIR + "/" + LEAST_REL_FEATURES_NAME + ".json"

##################
# --- MODELS --- #
##################

# Model dir
MODELS_DIR = MAIN_DIR + "/models"

# Model path
MODEL = MODELS_DIR + "/" + MODEL_NAME

#####################
# --- UTILITIES --- #
#####################

# Utilities dir
UTILITIES_DIR = MAIN_DIR + "/utilities"

###################
# --- RESULTS --- #
###################

# Results dir
RESULTS_DIR = MAIN_DIR + "/results/" + SPLITTING_METHOD

# Results path
MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + ".csv"
MODEL_ACCURACY_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_accuracy.csv"

In [4]:
# Suppression of warnings for better reading
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
if not LOCAL_RUNNING:
    # Install Spark and related dependencies
    !pip install pyspark
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

# Import files

In [6]:
# Import my files
import sys
sys.path.append(UTILITIES_DIR)

from imports import *
import utilities, parameters

importlib.reload(utilities)
importlib.reload(parameters)

<module 'parameters' from 'D:\\Documents/Repository/BDC/project/utilities\\parameters.py'>

# Create the pyspark session

In [7]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '109G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPricePrediction").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Loading dataset

In [8]:
# Load train / validation set into pyspark dataset objects
df = spark.read.load(DATASET_TRAIN_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [9]:
def dataset_info(dataset):
  # Print dataset
  dataset.show(3)

  # Get the number of rows
  num_rows = dataset.count()

  # Get the number of columns
  num_columns = len(dataset.columns)

  # Print the shape of the dataset
  print("Shape:", (num_rows, num_columns))

  # Print the schema of the dataset
  dataset.printSchema()

In [10]:
if SLOW_OPERATIONS:
  dataset_info(df)

+-------------------+---+------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+-------------------+------------------+--------------------+--------------------+------------------+-----------------+--------------------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|          timestamp| id|      market-price|      total-bitcoins|          market-cap|        trade-volume|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|          hash-rate|        difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|   n-transactions|estimated-transaction-volume-usd|      rate-of-change|        sma-5-days|        sma-7-days|       sma-10-days|       sma-20-days|       sma-50-days|      sma-100-days| next-market-price|
+-----

# Loading features

In [11]:
# Loading all the features
with open(ALL_FEATURES, "r") as f:
    ALL_FEATURES = json.load(f)
print(ALL_FEATURES)

['market-price', 'market-cap', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days', 'trade-volume', 'sma-100-days', 'n-unique-addresses', 'transaction-fees-usd', 'total-bitcoins', 'n-transactions-total', 'blocks-size', 'sma-50-days', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions', 'rate-of-change']


In [12]:
# Loading the most relevant features
with open(MOST_REL_FEATURES, "r") as f:
    MOST_REL_FEATURES = json.load(f)
print(MOST_REL_FEATURES)

['market-price', 'market-cap', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days', 'trade-volume']


In [13]:
# Loading least relevant features
with open(LEAST_REL_FEATURES, "r") as f:
    LEAST_REL_FEATURES = json.load(f)
print(LEAST_REL_FEATURES)

['sma-100-days', 'n-unique-addresses', 'transaction-fees-usd', 'total-bitcoins', 'n-transactions-total', 'blocks-size', 'sma-50-days', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions', 'rate-of-change']


# Model train / validation
In order to train and validate the model I'll try several approaches:
- **Default without normalization:** Make predictions using the chosen base model
- **Default with normalization:** Like the previous one but features are normalized

At this point, the features that gave on average the most satisfactory results (for each model) are chosen and proceeded with:

- **Hyperparameter tuning:** Researching the best parameters to use
- **Cross Validation:** Validate the performance of the model with the chosen parameters

If the final results are satisfactory, the model will be trained on the whole train / validation set and saved to later make predictions on the test set.

---

For each approach the train / validation set will be split according to the chosen splitting method (in order to figure out which one works best for our problem):

- **Block time series splits:** Involves dividing the time series into blocks of equal length, and then using each block as a separate fold for cross-validation.

    ![block-splits.png](https://drive.google.com/uc?id=1SPT133HO1VdWYZZv6GeknFY3xX3T2tvL)

- **Walk forward time series splits:** Involves using a sliding window approach to create the training and validation sets for each fold. The model is trained on a fixed window of historical data, and then validated on the next observation in the time series. This process is repeated for each subsequent observation, with the window sliding forward one step at a time. 

    ![walk-forward-splits.png](https://drive.google.com/uc?id=1SNdq-kjbv4MXtdBj3EOJ2dmQpbbPStJi)

- **Single time series split:** Involves dividing the time series considering as validation set a narrow period of time and as train set everything that happened before this period, in such a way as to best benefit from the trend in the short term.

    ![single-split.png](https://drive.google.com/uc?id=1SODyQLolK4zn9lFGnNaqnMBZrHn3OsVn)

In [14]:
# Get splitting parameters based on the choosen splitting method
splitting_info = parameters.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'walk_forward_splits',
 'min_obser': 20000,
 'sliding_window': 5000}

## Default
The train / validation set will be splitted based on the splitting method chosen so that the model performance can be seen without any tuning by using different features (normalized and non)

### Without normalization

In [15]:
# Define model and features type
MODEL_TYPE = "default"
FEATURES_NORMALIZATION = False

In [16]:
# Get default parameters
params = parameters.get_defaults_model_params(MODEL_NAME)
params

{'maxIter': [100], 'regParam': [0.0], 'elasticNetParam': [0.0]}

In [17]:
# Make predictions by using all the features
default_res_all, default_pred_all = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, ALL_FEATURES, ALL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_res_all

{'Model': 'LinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 1.544627280293475, 'MSE': 2.385873435026818, 'MAE': 1.273310205472029, 'MAPE': 0.00014114690073462725, 'R2': 0.999996862414024, 'Adjusted_R2': 0.9999968599014426, 'Time': 3.785831928253174}
{'Model': 'LinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 8.733650369761847, 'MSE': 76.27664878124125, 'MAE': 7.103079685648559, 'MAPE': 0.0007416044965220376, 'R2': 0.9996874026465642, 'Adjusted_R2': 0.9996871523183533, 'Time': 1.245237112045288}
{'Model': 'LinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 2.8579445901294545, 

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,walk_forward_splits,all_features,1,"(20000, 5000)","[100, 0.0, 0.0]",1.544627,2.385873,1.273310,0.000141,0.999997,0.999997,3.785832
1,LinearRegression,default,walk_forward_splits,all_features,2,"(20000, 5000)","[100, 0.0, 0.0]",8.733650,76.276649,7.103080,0.000742,0.999687,0.999687,1.245237
2,LinearRegression,default,walk_forward_splits,all_features,3,"(20000, 5000)","[100, 0.0, 0.0]",2.857945,8.167847,2.461786,0.000223,0.999976,0.999976,1.215742
3,LinearRegression,default,walk_forward_splits,all_features,4,"(20000, 5000)","[100, 0.0, 0.0]",38.353223,1470.969715,32.770936,0.002507,0.999486,0.999486,1.452628
4,LinearRegression,default,walk_forward_splits,all_features,5,"(20000, 5000)","[100, 0.0, 0.0]",4.295739,18.453374,2.983149,0.000137,0.999999,0.999999,1.134341
5,LinearRegression,default,walk_forward_splits,all_features,6,"(20000, 5000)","[100, 0.0, 0.0]",60.669631,3680.804079,46.518445,0.001165,0.999933,0.999933,1.040624
6,LinearRegression,default,walk_forward_splits,all_features,7,"(20000, 5000)","[100, 0.0, 0.0]",131.365094,17256.787947,117.719095,0.002057,0.999197,0.999196,0.987767
7,LinearRegression,default,walk_forward_splits,all_features,8,"(20000, 5000)","[100, 0.0, 0.0]",126.757007,16067.338796,115.552274,0.002756,0.999789,0.999789,0.937765
8,LinearRegression,default,walk_forward_splits,all_features,9,"(20000, 5000)","[100, 0.0, 0.0]",37.718249,1422.666275,31.325168,0.000909,0.999805,0.999805,0.807685
9,LinearRegression,default,walk_forward_splits,all_features,10,"(20000, 5000)","[100, 0.0, 0.0]",4.517135,20.404508,3.753351,0.000080,0.999998,0.999998,0.927839


In [18]:
# Make predictions by using the most relevant features
default_res_most_rel, default_pred_most_rel = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, MOST_REL_FEATURES, MOST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_res_most_rel

{'Model': 'LinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 2.7035860182874094, 'MSE': 7.309377358279168, 'MAE': 1.978041358383934, 'MAPE': 0.00022433394842406065, 'R2': 0.9999903876712167, 'Adjusted_R2': 0.9999903799736561, 'Time': 0.589543342590332}
{'Model': 'LinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 2.9811151144183046, 'MSE': 8.887047325413262, 'MAE': 2.4750849264021966, 'MAPE': 0.0002581978181830562, 'R2': 0.9999635790570486, 'Adjusted_R2': 0.9999635498911283, 'Time': 0.6610531806945801}
{'Model': 'LinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,walk_forward_splits,most_rel_features,1,"(20000, 5000)","[100, 0.0, 0.0]",2.703586,7.309377,1.978041,0.000224,0.999990,0.999990,0.589543
1,LinearRegression,default,walk_forward_splits,most_rel_features,2,"(20000, 5000)","[100, 0.0, 0.0]",2.981115,8.887047,2.475085,0.000258,0.999964,0.999964,0.661053
2,LinearRegression,default,walk_forward_splits,most_rel_features,3,"(20000, 5000)","[100, 0.0, 0.0]",3.175634,10.084649,2.240039,0.000199,0.999970,0.999970,0.523204
3,LinearRegression,default,walk_forward_splits,most_rel_features,4,"(20000, 5000)","[100, 0.0, 0.0]",4.508862,20.329841,3.493285,0.000272,0.999993,0.999993,0.540307
4,LinearRegression,default,walk_forward_splits,most_rel_features,5,"(20000, 5000)","[100, 0.0, 0.0]",7.115173,50.625691,5.281364,0.000253,0.999997,0.999997,0.528553
5,LinearRegression,default,walk_forward_splits,most_rel_features,6,"(20000, 5000)","[100, 0.0, 0.0]",18.688065,349.243771,14.642597,0.000369,0.999994,0.999994,0.475532
6,LinearRegression,default,walk_forward_splits,most_rel_features,7,"(20000, 5000)","[100, 0.0, 0.0]",20.640611,426.034839,16.787039,0.000305,0.999980,0.999980,0.507025
7,LinearRegression,default,walk_forward_splits,most_rel_features,8,"(20000, 5000)","[100, 0.0, 0.0]",24.218816,586.551070,17.598453,0.000385,0.999992,0.999992,0.457965
8,LinearRegression,default,walk_forward_splits,most_rel_features,9,"(20000, 5000)","[100, 0.0, 0.0]",17.236187,297.086140,14.823408,0.000434,0.999959,0.999959,0.564592
9,LinearRegression,default,walk_forward_splits,most_rel_features,10,"(20000, 5000)","[100, 0.0, 0.0]",17.137014,293.677260,12.950529,0.000281,0.999968,0.999968,0.494964


In [19]:
# Make predictions by using the least relevant features
default_res_least_rel, default_pred_least_rel = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, LEAST_REL_FEATURES, LEAST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_res_least_rel

{'Model': 'LinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 4754.974644396952, 'MSE': 22609783.868857913, 'MAE': 3192.911610339546, 'MAPE': 0.34431224087814793, 'R2': -28.733404859884462, 'Adjusted_R2': -28.757215394306794, 'Time': 0.4779326915740967}
{'Model': 'LinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 897.9342496159495, 'MSE': 806285.9166333583, 'MAE': 784.3428907393667, 'MAPE': 0.08331318160350913, 'R2': -2.3043250808648565, 'Adjusted_R2': -2.3069711870357192, 'Time': 0.6789693832397461}
{'Model': 'LinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE':

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,walk_forward_splits,least_rel_features,1,"(20000, 5000)","[100, 0.0, 0.0]",4754.974644,2.260978e+07,3192.911610,0.344312,-28.733405,-28.757215,0.477933
1,LinearRegression,default,walk_forward_splits,least_rel_features,2,"(20000, 5000)","[100, 0.0, 0.0]",897.934250,8.062859e+05,784.342891,0.083313,-2.304325,-2.306971,0.678969
2,LinearRegression,default,walk_forward_splits,least_rel_features,3,"(20000, 5000)","[100, 0.0, 0.0]",1547.495535,2.394742e+06,1342.578320,0.116819,-6.062510,-6.068166,0.553973
3,LinearRegression,default,walk_forward_splits,least_rel_features,4,"(20000, 5000)","[100, 0.0, 0.0]",1452.227756,2.108965e+06,1146.471861,0.092394,0.263278,0.262688,0.556751
4,LinearRegression,default,walk_forward_splits,least_rel_features,5,"(20000, 5000)","[100, 0.0, 0.0]",4348.085446,1.890585e+07,3883.268528,0.178752,-0.180427,-0.181373,0.527152
5,LinearRegression,default,walk_forward_splits,least_rel_features,6,"(20000, 5000)","[100, 0.0, 0.0]",7678.317162,5.895655e+07,6758.999458,0.172630,-0.067856,-0.068711,0.547221
6,LinearRegression,default,walk_forward_splits,least_rel_features,7,"(20000, 5000)","[100, 0.0, 0.0]",15265.911389,2.330481e+08,12180.818999,0.211317,-9.844444,-9.853128,0.772388
7,LinearRegression,default,walk_forward_splits,least_rel_features,8,"(20000, 5000)","[100, 0.0, 0.0]",17363.034617,3.014750e+08,16133.424111,0.377951,-2.951117,-2.954281,0.580067
8,LinearRegression,default,walk_forward_splits,least_rel_features,9,"(20000, 5000)","[100, 0.0, 0.0]",6643.073422,4.413042e+07,5821.663565,0.165155,-5.053438,-5.058286,0.731778
9,LinearRegression,default,walk_forward_splits,least_rel_features,10,"(20000, 5000)","[100, 0.0, 0.0]",3675.196850,1.350707e+07,3246.046446,0.069499,-0.481913,-0.483099,0.565215


### With normalization

In [20]:
# Define model and features type
MODEL_TYPE = "default_norm"
FEATURES_NORMALIZATION = True

In [21]:
# Make predictions by using all the features
default_norm_res_all, default_norm_pred_all = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, ALL_FEATURES, ALL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_norm_res_all

{'Model': 'LinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 299.5183036611932, 'MSE': 89711.21422807875, 'MAE': 230.59807561212094, 'MAPE': 0.0247420935495992, 'R2': 0.8820236465511144, 'Adjusted_R2': 0.881929170992797, 'Time': 0.7329802513122559}
{'Model': 'LinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 181.52287817012402, 'MSE': 32950.55529916569, 'MAE': 155.1784703544043, 'MAPE': 0.016262852373327533, 'R2': 0.8649618651928306, 'Adjusted_R2': 0.8648537265463383, 'Time': 0.7529969215393066}
{'Model': 'LinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 244.29

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,walk_forward_splits,all_features,1,"(20000, 5000)","[100, 0.0, 0.0]",299.518304,8.971121e+04,230.598076,0.024742,0.882024,0.881929,0.732980
1,LinearRegression,default_norm,walk_forward_splits,all_features,2,"(20000, 5000)","[100, 0.0, 0.0]",181.522878,3.295056e+04,155.178470,0.016263,0.864962,0.864854,0.752997
2,LinearRegression,default_norm,walk_forward_splits,all_features,3,"(20000, 5000)","[100, 0.0, 0.0]",244.297704,5.968137e+04,222.725487,0.019680,0.823989,0.823848,0.781152
3,LinearRegression,default_norm,walk_forward_splits,all_features,4,"(20000, 5000)","[100, 0.0, 0.0]",1048.578227,1.099516e+06,833.248148,0.062857,0.615907,0.615600,0.645350
4,LinearRegression,default_norm,walk_forward_splits,all_features,5,"(20000, 5000)","[100, 0.0, 0.0]",2789.448428,7.781023e+06,1874.231086,0.077140,0.514175,0.513786,0.654642
5,LinearRegression,default_norm,walk_forward_splits,all_features,6,"(20000, 5000)","[100, 0.0, 0.0]",2267.060731,5.139564e+06,1864.413079,0.042806,0.906909,0.906835,0.735104
6,LinearRegression,default_norm,walk_forward_splits,all_features,7,"(20000, 5000)","[100, 0.0, 0.0]",1754.289928,3.077533e+06,1634.029552,0.028995,0.856793,0.856678,0.695069
7,LinearRegression,default_norm,walk_forward_splits,all_features,8,"(20000, 5000)","[100, 0.0, 0.0]",14651.669469,2.146714e+08,12395.190374,0.299163,-1.813474,-1.815727,0.713956
8,LinearRegression,default_norm,walk_forward_splits,all_features,9,"(20000, 5000)","[100, 0.0, 0.0]",10287.500767,1.058327e+08,8541.425107,0.250873,-13.517231,-13.528856,0.698677
9,LinearRegression,default_norm,walk_forward_splits,all_features,10,"(20000, 5000)","[100, 0.0, 0.0]",5046.131038,2.546344e+07,4361.225320,0.092077,-1.793692,-1.795929,1.016931


In [22]:
# Make predictions by using the most relevant features
default_norm_res_most_rel, default_norm_pred_most_rel = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, MOST_REL_FEATURES, MOST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_norm_res_most_rel

{'Model': 'LinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 179.4242395225593, 'MSE': 32193.05772824873, 'MAE': 149.97306037275382, 'MAPE': 0.01711879030498661, 'R2': 0.9576639376712448, 'Adjusted_R2': 0.9576300349186292, 'Time': 0.46921491622924805}
{'Model': 'LinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 710.776803948509, 'MSE': 505203.66503125726, 'MAE': 664.1594212257036, 'MAPE': 0.0708664597878427, 'R2': -1.0704282523971407, 'Adjusted_R2': -1.0720862529996609, 'Time': 0.5829601287841797}
{'Model': 'LinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,walk_forward_splits,most_rel_features,1,"(20000, 5000)","[100, 0.0, 0.0]",179.424240,3.219306e+04,149.973060,0.017119,0.957664,0.957630,0.469215
1,LinearRegression,default_norm,walk_forward_splits,most_rel_features,2,"(20000, 5000)","[100, 0.0, 0.0]",710.776804,5.052037e+05,664.159421,0.070866,-1.070428,-1.072086,0.582960
2,LinearRegression,default_norm,walk_forward_splits,most_rel_features,3,"(20000, 5000)","[100, 0.0, 0.0]",638.856889,4.081381e+05,584.930181,0.051348,-0.203670,-0.204634,0.540057
3,LinearRegression,default_norm,walk_forward_splits,most_rel_features,4,"(20000, 5000)","[100, 0.0, 0.0]",1495.049703,2.235174e+06,1058.281627,0.076646,0.219190,0.218565,0.521203
4,LinearRegression,default_norm,walk_forward_splits,most_rel_features,5,"(20000, 5000)","[100, 0.0, 0.0]",6024.816108,3.629841e+07,5188.731281,0.232483,-1.266369,-1.268184,0.503513
5,LinearRegression,default_norm,walk_forward_splits,most_rel_features,6,"(20000, 5000)","[100, 0.0, 0.0]",13298.057034,1.768383e+08,11460.739599,0.266408,-2.203000,-2.205565,0.465686
6,LinearRegression,default_norm,walk_forward_splits,most_rel_features,7,"(20000, 5000)","[100, 0.0, 0.0]",8877.709537,7.881373e+07,8033.298215,0.139733,-2.667446,-2.670383,0.493252
7,LinearRegression,default_norm,walk_forward_splits,most_rel_features,8,"(20000, 5000)","[100, 0.0, 0.0]",23602.484662,5.570773e+08,19833.484983,0.490896,-6.301029,-6.306875,0.494289
8,LinearRegression,default_norm,walk_forward_splits,most_rel_features,9,"(20000, 5000)","[100, 0.0, 0.0]",27675.720998,7.659455e+08,23804.234639,0.698698,-104.065930,-104.150067,0.500513
9,LinearRegression,default_norm,walk_forward_splits,most_rel_features,10,"(20000, 5000)","[100, 0.0, 0.0]",45235.219560,2.046225e+09,39721.666494,0.846130,-223.499217,-223.678996,0.549258


In [23]:
# Make predictions by using the least relevant features
default_norm_res_least_rel, default_norm_pred_least_rel = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, LEAST_REL_FEATURES, LEAST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_norm_res_least_rel

{'Model': 'LinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 1756.4831869776199, 'MSE': 3085233.186135056, 'MAE': 1402.4758833445821, 'MAPE': 0.15411337846067177, 'R2': -3.0572916549130493, 'Adjusted_R2': -3.0605407373193865, 'Time': 0.5943875312805176}
{'Model': 'LinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.0, 0.0], 'RMSE': 1867.17753237837, 'MSE': 3486351.937418579, 'MAE': 1661.2637770571616, 'MAPE': 0.17498369706848763, 'R2': -13.287785399546415, 'Adjusted_R2': -13.299227069536043, 'Time': 0.6219887733459473}
{'Model': 'LinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [100, 0.

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,walk_forward_splits,least_rel_features,1,"(20000, 5000)","[100, 0.0, 0.0]",1756.483187,3.085233e+06,1402.475883,0.154113,-3.057292,-3.060541,0.594388
1,LinearRegression,default_norm,walk_forward_splits,least_rel_features,2,"(20000, 5000)","[100, 0.0, 0.0]",1867.177532,3.486352e+06,1661.263777,0.174984,-13.287785,-13.299227,0.621989
2,LinearRegression,default_norm,walk_forward_splits,least_rel_features,3,"(20000, 5000)","[100, 0.0, 0.0]",1734.491688,3.008461e+06,1586.107059,0.139288,-7.872474,-7.879579,0.609469
3,LinearRegression,default_norm,walk_forward_splits,least_rel_features,4,"(20000, 5000)","[100, 0.0, 0.0]",1171.285013,1.371909e+06,888.976633,0.071319,0.520753,0.520369,0.501627
4,LinearRegression,default_norm,walk_forward_splits,least_rel_features,5,"(20000, 5000)","[100, 0.0, 0.0]",7176.898909,5.150788e+07,6417.948973,0.292410,-2.216005,-2.218581,0.522749
5,LinearRegression,default_norm,walk_forward_splits,least_rel_features,6,"(20000, 5000)","[100, 0.0, 0.0]",5506.743922,3.032423e+07,4660.171576,0.124021,0.450750,0.450310,0.561138
6,LinearRegression,default_norm,walk_forward_splits,least_rel_features,7,"(20000, 5000)","[100, 0.0, 0.0]",12115.514130,1.467857e+08,9920.992361,0.172725,-5.830390,-5.835860,0.554945
7,LinearRegression,default_norm,walk_forward_splits,least_rel_features,8,"(20000, 5000)","[100, 0.0, 0.0]",21992.792904,4.836829e+08,20773.156378,0.486942,-5.339126,-5.344202,0.578345
8,LinearRegression,default_norm,walk_forward_splits,least_rel_features,9,"(20000, 5000)","[100, 0.0, 0.0]",13792.022199,1.902199e+08,13217.934303,0.380769,-25.092754,-25.113649,0.525293
9,LinearRegression,default_norm,walk_forward_splits,least_rel_features,10,"(20000, 5000)","[100, 0.0, 0.0]",14472.507883,2.094535e+08,13998.992333,0.298235,-21.979947,-21.998349,0.557345


In [24]:
# Define model information and evaluators to show
model_info = ['Model', 'Type', 'Splitting', 'Features', 'Parameters']
evaluator_lst = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2', 'Adjusted_R2', 'Time']

In [25]:
# Define the results to show
default_comparison_lst = [default_res_all, default_res_most_rel, default_res_least_rel, default_norm_res_all, default_norm_res_most_rel, default_norm_res_least_rel]

# Show the comparison table
default_comparison_lst_df = pd.concat([utilities.model_comparison(results, model_info, evaluator_lst) for results in default_comparison_lst])
default_comparison_lst_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,walk_forward_splits,all_features,"[100, 0.0, 0.0]",22.956868,1.872029e+03,19.655343,0.000621,0.999838,0.999838,1.063374
0,LinearRegression,default,walk_forward_splits,most_rel_features,"[100, 0.0, 0.0]",11.143603,1.648257e+02,8.621713,0.000276,0.999976,0.999976,0.535132
0,LinearRegression,default,walk_forward_splits,least_rel_features,"[100, 0.0, 0.0]",12250.863053,4.187280e+08,9903.491425,0.287631,-25.769681,-25.791118,0.617274
0,LinearRegression,default_norm,walk_forward_splits,all_features,"[100, 0.0, 0.0]",2547.856094,1.836015e+07,2145.693637,0.062408,-0.505045,-0.506250,0.731094
0,LinearRegression,default_norm,walk_forward_splits,most_rel_features,"[100, 0.0, 0.0]",9119.947927,1.908665e+08,7993.093986,0.240692,-28.992510,-29.016528,0.504287
0,LinearRegression,default_norm,walk_forward_splits,least_rel_features,"[100, 0.0, 0.0]",9279.897142,1.434210e+08,8258.956310,0.263110,-23.694272,-23.714047,0.577032


In [26]:
# Save the best default model results and predicitons
best_default_results = default_norm_res_all
best_default_predictions = default_norm_pred_all

## Tuned
Once the features and execution method are selected, the model will undergo hyperparameter tuning and cross validation to find the best configuration. 

### Hyperparameter tuning

In [27]:
# Select the type of feature to be used
MODEL_TYPE = "hyp_tuning"
CHOSEN_FEATURES = ALL_FEATURES
CHOSEN_FEATURES_LABEL = ALL_FEATURES_NAME
FEATURES_NORMALIZATION = True

In [28]:
# Get model grid parameters
params = parameters.get_model_grid_params(MODEL_NAME)
params

{'maxIter': [5, 10, 50, 80, 100],
 'regParam': array([0. , 0.2, 0.4, 0.6, 0.8]),
 'elasticNetParam': array([0. , 0.2, 0.4, 0.6, 0.8])}

In [29]:
# Perform hyperparameter tuning
hyp_res = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
hyp_res

{'Model': 'LinearRegression', 'Type': 'hyp_tuning', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [5, 0.8, 0.0], 'RMSE': 129.1342199206789, 'MSE': 16675.646754522262, 'MAE': 101.15871759404122, 'MAPE': 0.010876283004878152, 'R2': 0.9780703893885707, 'Adjusted_R2': 0.9780528281388319, 'Time': 0.20607542991638184}
{'Model': 'LinearRegression', 'Type': 'hyp_tuning', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [5, 0.4, 0.0], 'RMSE': 45.212804512630086, 'MSE': 2044.1976918973037, 'MAE': 38.425852434745615, 'MAPE': 0.004049016571569798, 'R2': 0.9916224585296163, 'Adjusted_R2': 0.9916157497876981, 'Time': 0.17215633392333984}
{'Model': 'LinearRegression', 'Type': 'hyp_tuning', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [50, 0.6, 0.6], 'RMSE': 153.27237

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,hyp_tuning,walk_forward_splits,all_features,1,"(20000, 5000)","[5, 0.8, 0.0]",129.134220,1.667565e+04,101.158718,0.010876,0.978070,0.978053,0.206075
1,LinearRegression,hyp_tuning,walk_forward_splits,all_features,2,"(20000, 5000)","[5, 0.4, 0.0]",45.212805,2.044198e+03,38.425852,0.004049,0.991622,0.991616,0.172156
2,LinearRegression,hyp_tuning,walk_forward_splits,all_features,3,"(20000, 5000)","[50, 0.6, 0.6]",153.272370,2.349242e+04,118.170872,0.010606,0.930717,0.930661,0.171417
3,LinearRegression,hyp_tuning,walk_forward_splits,all_features,4,"(20000, 5000)","[5, 0.2, 0.0]",257.646434,6.638168e+04,206.361095,0.016764,0.976811,0.976792,0.166476
4,LinearRegression,hyp_tuning,walk_forward_splits,all_features,5,"(20000, 5000)","[50, 0.6, 0.8]",343.038023,1.176751e+05,279.583556,0.014055,0.992653,0.992647,0.159223
5,LinearRegression,hyp_tuning,walk_forward_splits,all_features,6,"(20000, 5000)","[50, 0.2, 0.8]",1336.981845,1.787520e+06,994.425870,0.024500,0.967623,0.967597,0.161314
6,LinearRegression,hyp_tuning,walk_forward_splits,all_features,7,"(20000, 5000)","[100, 0.8, 0.4]",1043.285606,1.088445e+06,927.043046,0.016529,0.949351,0.949311,0.208547
7,LinearRegression,hyp_tuning,walk_forward_splits,all_features,8,"(20000, 5000)","[80, 0.4, 0.8]",3941.846863,1.553816e+07,3085.932458,0.071015,0.796358,0.796195,0.204932
8,LinearRegression,hyp_tuning,walk_forward_splits,all_features,9,"(20000, 5000)","[50, 0.8, 0.8]",1036.081409,1.073465e+06,761.083087,0.022098,0.852751,0.852633,0.150097
9,LinearRegression,hyp_tuning,walk_forward_splits,all_features,10,"(20000, 5000)","[80, 0.4, 0.4]",1482.513025,2.197845e+06,1197.904247,0.025950,0.758866,0.758673,0.167088


---
To select the best parameters to be used in the final model I assign a score to each value in the "Parameters" column based on the following criteria:
* Calculate the frequency of each unique value in the "Parameters" column.
* Normalize the frequencies to a scale of 0 to 1, where 1 represents the highest frequency.
* Calculate the split weight for each value in the "Parameters" column, where a higher split number corresponds to a higher weight.
* Normalize the split weights to a scale of 0 to 1, where 1 represents the highest split weight.
* Calculate the RMSE weight for each value in the "Parameters" column, where a lower RMSE value corresponds to a higher weight.
* Normalize the RMSE weights to a scale of 0 to 1, where 1 represents the highest RMSE weight.

Then calculate the overall score for each value in the "Parameters" column by combining the normalized frequency, split weight, and RMSE weight and take into consideration the parameters that have the highest score.

In [30]:
# Show parameters score
grouped_scores, best_params = parameters.choose_best_params(hyp_res)
grouped_scores

,Split weight,RMSE weight,Frequency weight,Final score
Parameters,,,,
"(5, 0.0, 0.0)",0.787879,0.844512,0.75,0.499030
"(5, 0.2, 0.0)",0.556818,0.862697,1.00,0.480365
"(5, 0.8, 0.0)",0.500000,0.881374,0.75,0.330515
"(80, 0.2, 0.6)",1.000000,0.925336,0.25,0.231334
"(100, 0.6, 0.6)",0.818182,0.922118,0.25,0.188615
"(100, 0.8, 0.6)",0.772727,0.842340,0.25,0.162725
"(100, 0.8, 0.8)",0.545455,0.789130,0.25,0.107609
"(50, 0.8, 0.8)",0.409091,0.737158,0.25,0.075391
"(80, 0.4, 0.4)",0.454545,0.623904,0.25,0.070898


In [31]:
# Print best parameters
print(f"Best parameters: {best_params}")

Best parameters: (5, 0.0, 0.0)


### Cross validation

In [32]:
MODEL_TYPE = "cross_val"

In [33]:
# Get tuned parameters
params = parameters.get_best_model_params(best_params, MODEL_NAME)
params

{'maxIter': [5], 'regParam': [0.0], 'elasticNetParam': [0.0]}

In [34]:
# Perform cross validation
cv_res, cv_pred = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
cv_res

{'Model': 'LinearRegression', 'Type': 'cross_val', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [5, 0.0, 0.0], 'RMSE': 560.0378136270248, 'MSE': 313642.35269213817, 'MAE': 453.6238618831982, 'MAPE': 0.04888796406402696, 'R2': 0.5875389562371307, 'Adjusted_R2': 0.5872086571029862, 'Time': 0.6674163341522217}
{'Model': 'LinearRegression', 'Type': 'cross_val', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [5, 0.0, 0.0], 'RMSE': 181.52287817012402, 'MSE': 32950.55529916569, 'MAE': 155.1784703544043, 'MAPE': 0.016262852373327533, 'R2': 0.8649618651928306, 'Adjusted_R2': 0.8648537265463383, 'Time': 0.5113675594329834}
{'Model': 'LinearRegression', 'Type': 'cross_val', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [5, 0.0, 0.0], 'RMSE': 620.0147523061988, 

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,cross_val,walk_forward_splits,all_features,1,"(20000, 5000)","[5, 0.0, 0.0]",560.037814,3.136424e+05,453.623862,0.048888,0.587539,0.587209,0.667416
1,LinearRegression,cross_val,walk_forward_splits,all_features,2,"(20000, 5000)","[5, 0.0, 0.0]",181.522878,3.295056e+04,155.178470,0.016263,0.864962,0.864854,0.511368
2,LinearRegression,cross_val,walk_forward_splits,all_features,3,"(20000, 5000)","[5, 0.0, 0.0]",620.014752,3.844183e+05,579.457984,0.051955,-0.133716,-0.134624,0.504413
3,LinearRegression,cross_val,walk_forward_splits,all_features,4,"(20000, 5000)","[5, 0.0, 0.0]",1048.578227,1.099516e+06,833.248148,0.062857,0.615907,0.615600,0.404925
4,LinearRegression,cross_val,walk_forward_splits,all_features,5,"(20000, 5000)","[5, 0.0, 0.0]",936.304063,8.766653e+05,829.503681,0.041520,0.945264,0.945220,0.491524
5,LinearRegression,cross_val,walk_forward_splits,all_features,6,"(20000, 5000)","[5, 0.0, 0.0]",1937.454575,3.753730e+06,1514.551505,0.038002,0.932010,0.931956,0.439323
6,LinearRegression,cross_val,walk_forward_splits,all_features,7,"(20000, 5000)","[5, 0.0, 0.0]",5451.074700,2.971422e+07,5178.829404,0.092238,-0.382694,-0.383801,0.544134
7,LinearRegression,cross_val,walk_forward_splits,all_features,8,"(20000, 5000)","[5, 0.0, 0.0]",14651.669469,2.146714e+08,12395.190374,0.299163,-1.813474,-1.815727,0.472726
8,LinearRegression,cross_val,walk_forward_splits,all_features,9,"(20000, 5000)","[5, 0.0, 0.0]",5774.030675,3.333943e+07,5242.498363,0.154258,-3.573221,-3.576883,0.591223
9,LinearRegression,cross_val,walk_forward_splits,all_features,10,"(20000, 5000)","[5, 0.0, 0.0]",5046.131038,2.546344e+07,4361.225320,0.092077,-1.793692,-1.795929,0.446009


In [35]:
# Define the results to show
tuned_comparison_lst = [cv_res]

# Show the comparison table
tuned_comparison_lst_df = pd.concat([utilities.model_comparison(results, model_info, evaluator_lst) for results in tuned_comparison_lst])
tuned_comparison_lst_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,cross_val,walk_forward_splits,all_features,"[5, 0.0, 0.0]",2617.062951,1.709796e+07,2281.635511,0.064379,-0.163743,-0.164674,0.474489


# Comparison table
Visualization of model performance at various stages of train / validation

In [36]:
# Concatenate final results into Pandas dataset
final_comparison_lst_df = pd.DataFrame(pd.concat([default_comparison_lst_df, tuned_comparison_lst_df], ignore_index=True))
final_comparison_lst_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default,walk_forward_splits,all_features,"[100, 0.0, 0.0]",22.956868,1.872029e+03,19.655343,0.000621,0.999838,0.999838,1.063374
1,LinearRegression,default,walk_forward_splits,most_rel_features,"[100, 0.0, 0.0]",11.143603,1.648257e+02,8.621713,0.000276,0.999976,0.999976,0.535132
2,LinearRegression,default,walk_forward_splits,least_rel_features,"[100, 0.0, 0.0]",12250.863053,4.187280e+08,9903.491425,0.287631,-25.769681,-25.791118,0.617274
3,LinearRegression,default_norm,walk_forward_splits,all_features,"[100, 0.0, 0.0]",2547.856094,1.836015e+07,2145.693637,0.062408,-0.505045,-0.506250,0.731094
4,LinearRegression,default_norm,walk_forward_splits,most_rel_features,"[100, 0.0, 0.0]",9119.947927,1.908665e+08,7993.093986,0.240692,-28.992510,-29.016528,0.504287
5,LinearRegression,default_norm,walk_forward_splits,least_rel_features,"[100, 0.0, 0.0]",9279.897142,1.434210e+08,8258.956310,0.263110,-23.694272,-23.714047,0.577032
6,LinearRegression,cross_val,walk_forward_splits,all_features,"[5, 0.0, 0.0]",2617.062951,1.709796e+07,2281.635511,0.064379,-0.163743,-0.164674,0.474489


# Model accuracy

Since predicting the price accurately is very difficult let's se how good the model is at predicting whether the price will go up or down. 

For each row in our predictions let's consider the actual market-price, next-market-price and our predicted next-market-price (prediction).
We compute whether the current prediction is correct (1) or not (0):

$$ 
prediction\_is\_correct
= 
\begin{cases}
0 \text{ if [(market-price > next-market-price) and (market-price < prediction)] or [(market-price < next-market-price) and (market-price > prediction)]} \\
1 \text{ if [(market-price > next-market-price) and (market-price > prediction)] or [(market-price < next-market-price) and (market-price < prediction)]}
\end{cases}
$$

After that we count the number of correct prediction:
$$ 
correct\_predictions
= 
\sum_{i=0}^{total\_rows} prediction\_is\_correct
$$

Finally we compute the percentage of accuracy of the model:
$$
\\ 
accuracy 
= 
(correct\_predictions / total\_rows) 
* 100
$$

In [37]:
# Convert the pandas dataset to a PySpark dataset
best_default_pred_spark = spark.createDataFrame(best_default_predictions)
validated_pred_spark = spark.createDataFrame(cv_pred)

# Compute model accuracy
default_accuracy = utilities.model_accuracy(best_default_pred_spark)
validated_accuracy = utilities.model_accuracy(validated_pred_spark)

# Saving accuracy data into dataframe
accuracy_data = {
    'Model': MODEL_NAME,
    'Features': CHOSEN_FEATURES_LABEL,
    'Splitting': SPLITTING_METHOD,
    'Accuracy (default)': default_accuracy,
    'Accuracy (validated)': validated_accuracy
}
accuracy_data_df = pd.DataFrame(accuracy_data, index=['Model'])

print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (default): {default_accuracy:.2f}%")
print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (validated): {validated_accuracy:.2f}%")

Percentage of correct predictions for LinearRegression with all_features and walk_forward_splits (default): 49.33%
Percentage of correct predictions for LinearRegression with all_features and walk_forward_splits (validated): 49.13%


# Saving final results


In [38]:
# Concatenate default and tuned results
default_tuned_results = [best_default_results, cv_res]
default_tuned_results_df = pd.concat([utilities.model_comparison(results, model_info, evaluator_lst) for results in default_tuned_results])
default_tuned_results_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,LinearRegression,default_norm,walk_forward_splits,all_features,"[100, 0.0, 0.0]",2547.856094,1.836015e+07,2145.693637,0.062408,-0.505045,-0.506250,0.731094
0,LinearRegression,cross_val,walk_forward_splits,all_features,"[5, 0.0, 0.0]",2617.062951,1.709796e+07,2281.635511,0.064379,-0.163743,-0.164674,0.474489


In [39]:
# Saving default and tuned results
default_tuned_results_df.to_csv(MODEL_RESULTS, index=False)

In [40]:
# Saving accuracy results
accuracy_data_df.to_csv(MODEL_ACCURACY_RESULTS, index=False)